In [4]:
import cv2
import numpy as np
import os

# Путь к изображениям
LEFT_PATH = '../data/calibrate/imgs/leftcamera'
RIGHT_PATH = '../data/calibrate/imgs/rightcamera'

# Чтение изображений
left_images = [cv2.imread(os.path.join(LEFT_PATH, fname)) for fname in os.listdir(LEFT_PATH)]
right_images = [cv2.imread(os.path.join(RIGHT_PATH, fname)) for fname in os.listdir(RIGHT_PATH)]

# Проверка количества изображений
assert len(left_images) == len(right_images), "Количество изображений в левом и правом каталогах не совпадает"

# Инициализация SIFT
sift = cv2.SIFT_create(nfeatures=5000, contrastThreshold=0.02, edgeThreshold=10, sigma=1.6)

# Списки для хранения всех ключевых точек и дескрипторов
all_left_keypoints = []
all_left_descriptors = []
all_right_keypoints = []
all_right_descriptors = []

# Поиск ключевых точек и дескрипторов на всех изображениях
for left_img, right_img in zip(left_images, right_images):
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # Обнаружение ключевых точек и вычисление дескрипторов
    left_kp, left_des = sift.detectAndCompute(left_gray, None)
    right_kp, right_des = sift.detectAndCompute(right_gray, None)

    all_left_keypoints.append(left_kp)
    all_left_descriptors.append(left_des)
    all_right_keypoints.append(right_kp)
    all_right_descriptors.append(right_des)

# Создание BFMatcher объекта для поиска соответствий между дескрипторами
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Перебор всех пар изображений
for i in range(len(all_left_descriptors)):
    # Поиск соответствий
    matches = bf.match(all_left_descriptors[i], all_right_descriptors[i])

    # Сортировка соответствий по расстоянию
    matches = sorted(matches, key=lambda x: x.distance)

    # Отображение соответствий
    img_matches = cv2.drawMatches(left_images[i], all_left_keypoints[i], right_images[i], all_right_keypoints[i], matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Вывод изображения с совпадениями
    img_matches = cv2.resize(img_matches,(1080,720))
    cv2.imshow(f'Matches {i+1}', img_matches)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Извлечение точек из совпадений
    src_pts = np.float32([all_left_keypoints[i][m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([all_right_keypoints[i][m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Вычисление фундаментальной матрицы
    F, mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.FM_RANSAC)

    # Фильтрация соответствий с использованием маски
    matchesMask = mask.ravel().tolist()

    # Вычисление гомографии
    H, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Вывод фундаментальной матрицы и гомографии
    print(f'Фундаментальная матрица для пары {i+1}:\n', F)
    print(f'Гомография для пары {i+1}:\n', H)

Фундаментальная матрица для пары 1:
 [[ 2.20749988e-07 -2.26423808e-08 -6.23526413e-04]
 [ 4.68388846e-08  6.74199864e-09 -1.51774595e-04]
 [-3.44000057e-04  1.84792851e-05  1.00000000e+00]]
Гомография для пары 1:
 [[-4.02597468e+00  3.61771056e+00  1.15838586e+03]
 [-5.00713756e+00  4.51428973e+00  1.39807705e+03]
 [-3.28410409e-03  2.93938146e-03  1.00000000e+00]]
Фундаментальная матрица для пары 2:
 [[ 4.42511988e-08 -6.65512834e-08 -4.66878194e-05]
 [ 4.87787371e-08  1.15963094e-07 -2.37767274e-04]
 [-2.69670824e-04 -3.21512232e-04  1.00000000e+00]]
Гомография для пары 2:
 [[-4.41119144e-01 -6.33920800e-01  1.78931139e+03]
 [-9.45086477e-01 -1.35920340e+00  3.83095229e+03]
 [-2.46708686e-04 -3.54837559e-04  1.00000000e+00]]
Фундаментальная матрица для пары 3:
 [[ 1.71304820e-07 -1.59873471e-07 -1.11957930e-04]
 [ 1.04901100e-06 -1.03915187e-06 -7.26209745e-04]
 [-1.47262227e-03  1.42995685e-03  1.00000000e+00]]
Гомография для пары 3:
 [[-1.78137826e+02  2.42597933e+02  3.58018896e+